In [29]:
import tifffile
import numpy as np
from tqdm import tqdm
from pathlib import Path
import os
import pandas as pd
import sys
sys.path.append(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\10_code\UTvsXCT-preprocessing')
import dbtools.dbtools as db
import dbtools.load as load
from preprocess_tools import io, sample_isolater

# Database conection

In [30]:
try:
    conn = db.connect()
    print("Connected to the database")
except Exception as error:
    print(error)

Connected to the database


# Data retrieval

We have to load the data from the database to get:

1. The file ids to use them as parent measurement

## Measurements file paths

In [31]:
original_paths = [Path(r"\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Panel PEGASO\lente\medidas\Airbus_Piramide1p16_17_21_22_23_12dB_DAC_10mm_20mms_lente_42mmdistance\Airbus_Piramide1p16_17_21_22_23_12dB_DAC_10mm_20mms_lente_42mmdistance_ascan320x187x448.tif")
]

measurements_table = db.relation_metadata('measurements','samples','sample_measurements')

parent_id_column = 'measurementtype_id_measurement'

saving_folder = Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Panel PEGASO\lente\probetas')

# for each original path, get the id_measurement where file_path_measurement is equal to the str of the original path

original_ids = []
original_measurementtype_ids = []
original_sample_names = []

for original_path in original_paths:
    original_path_str = str(original_path)
    original_id = measurements_table.loc[measurements_table['file_path_measurement'] == original_path_str, 'id_measurement'].values
    original_measurementtype_id = measurements_table.loc[measurements_table['file_path_measurement'] == original_path_str, parent_id_column].values[0]
    original_sample_name = measurements_table.loc[measurements_table['file_path_measurement'] == original_path_str, 'name_sample'].values
    original_ids.append(original_id)
    original_measurementtype_ids.append(original_measurementtype_id)
    original_sample_names.append(original_sample_name)
    print(f"Original path: {original_path_str}, ID: {original_id}", 
          f"Measurement type ID: {original_measurementtype_id}",
          f"Sample names: {original_sample_name}")

Original path: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Panel PEGASO\lente\medidas\Airbus_Piramide1p16_17_21_22_23_12dB_DAC_10mm_20mms_lente_42mmdistance\Airbus_Piramide1p16_17_21_22_23_12dB_DAC_10mm_20mms_lente_42mmdistance_ascan320x187x448.tif, ID: [349 349 349 349 349] Measurement type ID: 10 Sample names: ['1.16' '1.17' '1.21' '1.22' '1.23']


In [32]:
print('These are the samples present in the measurement:', original_sample_names)

These are the samples present in the measurement: [array(['1.16', '1.17', '1.21', '1.22', '1.23'], dtype=object)]


You have to set the order of the samples in the measurement from left to right.

In [33]:
samples_order = ['1.16','1.17', '1.21', '1.22', '1.23']
#assert that all original_sample_names are in samples_order
for original_sample_name in original_sample_names:
    assert original_sample_name[0] in samples_order, f"{original_sample_name[0]} not in samples_order"

## Functions

## Database loading

## main loop

For each file:

1. Load it

2. Separate the samples in it

3. Save each isolated sample

4. Save them to the database

In [34]:
for i in range(len(original_paths)):
    # Get the original path and ID
    original_path = original_paths[i]
    original_id = original_ids[i]
    original_measurementtype_id = original_measurementtype_ids[i]
    original_sample_name = original_sample_names[i]
    number_of_samples = len(original_sample_name)

    #load the volume
    volume = io.load_tif(original_path)

    #isolate the samples
    isolated_volumes = sample_isolater.isolate_samples_ut(volume, number_of_samples)

    for j,isolated_volume in enumerate(isolated_volumes):

        sample_name = samples_order[j]
        #substitute . with _
        sample_name = sample_name.replace('.', '_')

        folder_name = saving_folder / sample_name
        
        #save the volume
        #save it in the parent folder of it with the name of the sample
        if not folder_name.exists():
            folder_name.mkdir(parents=True, exist_ok=True)
        save_path = folder_name / f"{sample_name}.tif"
        print(f"Saving isolated volume for sample {original_sample_name[j]} in {save_path}")
        #save in the path
        io.save_tif(save_path, isolated_volume)

        height,widht,depth = isolated_volume.shape

        # #update the database
        load.load_ut_measurement(conn,str(save_path),int(original_measurementtype_ids[i]),height,widht,depth,str(isolated_volume.dtype),'tif','RF',['z','y','x'],[original_sample_name[j]],str(original_paths[i]),'Cropped to region containing only the sample')

Saving isolated volume for sample 1.16 in \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Panel PEGASO\lente\probetas\1_16\1_16.tif
UT measurement from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Panel PEGASO\lente\probetas\1_16\1_16.tif' loaded with ID: 374
Saving isolated volume for sample 1.17 in \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Panel PEGASO\lente\probetas\1_17\1_17.tif
UT measurement from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Panel PEGASO\lente\probetas\1_17\1_17.tif' loaded with ID: 375
Saving isolated volume for sample 1.21 in \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Panel PEGASO\lente\probetas\1_21\1_21.tif
UT measurement from '\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Panel PEGASO\lente\probetas\1_21\1_21.tif' loaded with ID: 376
Saving isolated volume for sample 1.22 in \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Panel PEGASO\